<img src="images/splash3.png">
Requests data for the past hour and inserts this into SQL. To be ran hourly.


<p><b>What you were doing:</b></p>
<p>&nbsp; &nbsp; possible issue: only fetches tweets with 140+ character length? </p>

## Setup

In [1]:
import sqlite3 as lite
import sys
import requests
import pandas as pd
import numpy as np
import datetime
import json
import tweepy

In [2]:
# Twitter stuff
CONSUMER_KEY = 'hiZmnSOV63GdqWPqW5MN533T1'# TWITTER_CONSUMER_TOKEN
CONSUMER_SECRET = 'GJa7snhSw8jDQxTVhvoHJyIyJZBjLQikAeuNL0Y3o0Aw61zuOr' # TWITTER_CONSUMER_SECRET
ACCESS_TOKEN = '933287551212904448-uWflGB32dNF8rbgmBxJk1ql5AqemO14' # TWITTER_ACCESS_TOKEN
ACCESS_SECRET = 'boUb7syKFxrCmaJN3T3SHsWu6rDQQfdPxertNIEnXR6X9' # TWITTER_ACCESS_SECRET

## Class & function definitions


In [5]:
class Coin:
    def __init__(self, symbol, comparison_symbol, limit, aggregate):
        self.symbol = symbol
        self.comparison_symbol = comparison_symbol
        self.limit = limit
        self.aggregate = aggregate
        
    def fetch_marketdata(self):    
        url = 'https://min-api.cryptocompare.com/data/histohour?fsym={}&tsym={}&limit={}&aggregate={}'\
                .format(self.symbol.upper(), self.comparison_symbol.upper(), self.limit, self.aggregate)       
        page = requests.get(url)
        data = page.json()['Data']
        df = pd.DataFrame(data)
        df['mid'] = df[["high", "low"]].mean(axis=1)
        df['timestamp'] = [datetime.datetime.fromtimestamp(d) for d in df.time]
        return df
    
    def fetch_tweets(self):
        # Create extractor object
        extractor = twitter_setup()
        # Create a tweet list
        #tweets = extractor.search(q ="${}".format(coin_instances[coin].symbol), count=20, tweet_mode="extended", lang="en")
        #print("[coin] = ", coin)
        #print("Searching for coin_instances[coin].symbol: ", coin_instances[coin].symbol)
        tweets = extractor.search(q ="${} {}".format(coin_instances[coin].symbol, "-url:binance -url:kucoin -url:ku-coin -url:discord"), 
                                  count=100, 
                                  tweet_mode="extended", lang="en")
        
        def fulltext(tweet):
            try:
                text = tweet.retweeted_status.full_text
                return(text)
            except:
                text = tweet.full_text
                return(text)

        df = pd.DataFrame(data=[fulltext(tweet) for tweet in tweets], columns=['Tweets'])
        
        df['time'] = np.array([tweet.created_at for tweet in tweets])
        df['time'] = df['time'].astype(np.int64) // 10**9 # Convert to unix
        df['ID']   = np.array([tweet.id for tweet in tweets])
        df['Len']  = np.array([len(tweet.full_text) for tweet in tweets])
        df['Reply_To_Status'] = np.array([tweet.in_reply_to_user_id for tweet in tweets])
        df['Reply_To_Name'] = np.array([tweet.in_reply_to_screen_name for tweet in tweets])
        df['Coordinates'] = np.array([tweet.coordinates for tweet in tweets])
        df['Retweet_Count'] = np.array([tweet.retweet_count for tweet in tweets])
        df['User_Verified'] = np.array([tweet.user.verified for tweet in tweets])
        df['User_Location'] = np.array([tweet.user.location for tweet in tweets])
        df['User_Followers_Count'] = np.array([tweet.user.followers_count for tweet in tweets])
        df['User_Favorite_Count'] = np.array([tweet.user.favourites_count for tweet in tweets])
        df['Source'] = np.array([tweet.source for tweet in tweets])
        df['Likes']  = np.array([tweet.favorite_count for tweet in tweets])
        df['Favorite_Count'] = np.array([tweet.favorite_count for tweet in tweets])
        df['RTs'] = np.array([tweet.retweet_count for tweet in tweets])        
        return df
        
def twitter_setup():
    """
    Utility function to setup the Twitter's API
    with our access keys provided.
    """
    # Authentication and access using keys
    auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

    # Return API with authentication
    api = tweepy.API(auth)
    return api    
    
    
def market_to_db(market_data):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB

    
    with con: #Dump to SQL
        try:
            date = pd.read_sql_query("SELECT time FROM {} ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol), con)
            latest_db_entry = date['time'].values[0]
        except Exception as e:
            print("Skipping coin", coin_instances[coin].symbol, ", Error: ")
            print(e)
            latest_db_entry = 0 # Probably no table exists, set 
            
        if latest_db_entry < market_data.iloc[0]['time']:
            #print(latest_db_entry, "is lower than", market_data.iloc[0]['time'], "<- Time to write!") # Debug
            market_data.to_sql(name=coin_instances[coin].symbol, con=con, if_exists='append', index=True)
            #print("wrote to DB for coin", coin_instances[coin].symbol) # Debug 
        else:
            pass
            #print(latest_db_entry, "(latest db entry) is the same or higher than", market_data.iloc[0]['time'], "(fetched data), skipping write for", coinlist[coin])
            
def tweets_to_db(df_tweets):
    con = None
    con = lite.connect('/hdd/db/crypto.db') #Connect to DB
    
    with con: #Dump to SQL
        try:
            date = pd.read_sql_query("SELECT time FROM {}_tweets ORDER BY time DESC LIMIT 1".format(coin_instances[coin].symbol), con)
            latest_db_entry = date['time'].values[0] #Get last date
        except Exception as e:
            print("Skipping coin", coin_instances[coin].symbol, ", Error: ")
            print(e)
            latest_db_entry = 0 # Probably no table exists, set 
        
        df_tweets = df_tweets.loc[df_tweets['time'] > latest_db_entry] #Slice df to only include new tweets
        
        if(len(df_tweets) > 0): # If there is something to write 
            df_tweets.to_sql(name="{}_tweets".format(coin_instances[coin].symbol), con=con, if_exists='append', index=True)
            #print("Going to write {} tweets to {}_tweets".format(len(df_tweets), coin_instances[coin].symbol))

## Fetch data & store in db

In [8]:
fh = open("/hdd/db/tracked_coins.json", 'r')
coinlist = json.load(fh)
coin_instances = [Coin(coinlist[coin], 'BTC','1','1') for coin in range(len(coinlist))] # Make Coin objects. Take all coins in coinlist, convert to BTC, poll 1 entry and aggregate per 1-hour period.

#Get data
for coin in range(len(coin_instances)):
    try:
        market_data = coin_instances[coin].fetch_marketdata() # Get coindata
        df_tweets = coin_instances[coin].fetch_tweets() # Get tweets
        market_data = market_data.drop(market_data.index[len(market_data)-1]) # Only keep the latest timestamped column
        market_to_db(market_data)
        tweets_to_db(df_tweets)
    except Exception as e:
        print("Skipping coin", coin_instances[coin].symbol, ", Error: ")
        print(e)

Skipping coin BTCP , Error: 
Execution failed on sql 'SELECT time FROM BTCP ORDER BY time DESC LIMIT 1': no such table: BTCP
Skipping coin BTCP , Error: 
Execution failed on sql 'SELECT time FROM BTCP_tweets ORDER BY time DESC LIMIT 1': no such table: BTCP_tweets
Skipping coin LOOM , Error: 
Execution failed on sql 'SELECT time FROM LOOM ORDER BY time DESC LIMIT 1': no such table: LOOM
Skipping coin LOOM , Error: 
Execution failed on sql 'SELECT time FROM LOOM_tweets ORDER BY time DESC LIMIT 1': no such table: LOOM_tweets
Skipping coin WAN , Error: 
Execution failed on sql 'SELECT time FROM WAN ORDER BY time DESC LIMIT 1': no such table: WAN
Skipping coin WAN , Error: 
Execution failed on sql 'SELECT time FROM WAN_tweets ORDER BY time DESC LIMIT 1': no such table: WAN_tweets
Skipping coin MITH , Error: 
Execution failed on sql 'SELECT time FROM MITH ORDER BY time DESC LIMIT 1': no such table: MITH
Skipping coin MITH , Error: 
Execution failed on sql 'SELECT time FROM MITH_tweets ORDER 

In [305]:
# ### JUST SOME PRINT LINES ###
# coin = 95
# print("Coin: ", coinlist[95])

# market_data = coin_instances[95].fetch_marketdata() # Get coindata
# print(market_data)

# df_tweets = coin_instances[95].fetch_tweets() # Get tweets

Coin:  DOGE
          close          high           low          open        time  \
0  3.900000e-07  4.000000e-07  3.900000e-07  3.900000e-07  1531123200   
1  3.900000e-07  3.900000e-07  3.900000e-07  3.900000e-07  1531126800   

    volumefrom  volumeto           mid           timestamp  
0  11980491.43      4.65  3.950000e-07 2018-07-09 10:00:00  
1         0.00      0.00  3.900000e-07 2018-07-09 11:00:00  


In [306]:
# print("\n ------------- Random tweet -----------------:\n")
# print(df_tweets['Tweets'].iloc[1])


 ------------- Random tweet -----------------:

this channel called $DGB before it made 6x. they only share high reward, low risk signals. join:

https://t.co/T5egWYl250

$BTC $ETH $ETC $BCH $LTC $XRP $DASH $BTG $XLM $XMR $ZEC $ADA $SNT $NEO $NXT $OMG $POWR $VTC $VOX $XEM $LSK $DGB $DOGE $XVG $GRS $AMP $ZCL $DRGN $DCR 79548
